# Colbert Recall

In [1]:
import os
import polars as pl
import polars.selectors as cs
import pandas as pd
import pyarrow as pa
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm

from rank_bm25 import BM25Okapi
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert.data import Queries, Collection
from colbert import Indexer, Searcher

pl.Config(fmt_str_lengths=2000);

In [13]:
questions = pl.concat([
    pl.read_csv('./data/daniel_train_aug_19_200-300.csv'),
    pl.read_csv('./data/daniel_train_aug_19_400-500.csv')
])
questions.drop_in_place('incorrect_6')
questions.shape
questions[:1]

question,correct,incorrect_1,incorrect_2,incorrect_3,incorrect_4,incorrect_5,title,section_title
str,str,str,str,str,str,str,str,str
"""In mathematics, what is an iterated function?""","""An iterated function is a function obtained by composing another function with itself a certain number of times.""","""An iterated function is a function that can only be applied once.""","""An iterated function is a function that operates on a set of fixed points.""","""An iterated function is a function that guarantees convergence acceleration.""","""An iterated function is a function that produces quadratic convergence.""","""An iterated function is a function that involves the Aitken method.""","""Iterated function""","""Fixed points"""


In [14]:
nbits = 2   # encode each dimension with 2 bits
doc_maxlen = 512   # truncate passages at 300 tokens
dim = 512 # default is 128

checkpoint = './checkpoints/colbertv2.0'
experiment = 'wiki-science'
indexer_name = f"wiki_pages_index_{nbits}bits_{dim}dim"

# os.environ['COLBERT_LOAD_TORCH_EXTENSION_VERBOSE'] = 'True'

config = ColBERTConfig(
    doc_maxlen=doc_maxlen,
    nbits=nbits,
    dim=dim
)